In [ ]:
%cd
%tensorflow_version 1.x

import os
import zipfile 
import tensorflow as tf

print(tf.__version__)

!pip install q keras==2.1.0
!git clone --quiet https://github.com/WimFlorijn/instance-segmentation-maskrcnn.git

%cd ~/instance-segmentation-maskrcnn

In [ ]:
!pip install -q PyDrive

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
dataset_dir = '/tmp/traindata'

zip_ref = zipfile.ZipFile('/content/gdrive/My Drive/train_data.zip', 'r')
zip_ref.extractall(dataset_dir)
zip_ref.close()

In [ ]:
import os
import re
import sys
import cv2
import math
import time
import keras
import random
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath('../../')

# Import Mask RCNN
sys.path.append(ROOT_DIR)
import mrcnn.model as modellib

from mrcnn import utils
from mrcnn import visualize
from mrcnn.model import log
from mrcnn.config import Config

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = '/tmp/snapshot'

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """ Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.

    Change the default size attribute to control the size
    of rendered images
    """

    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))

    return ax

In [ ]:
class SketchDataset(utils.Dataset):

    def load_sketch(self, image_ids):
        self.add_class("sketch", 1, "measurement")
        self.add_class("sketch", 2, "parcel_number")

        for image_id in image_ids:
          path = os.path.join(dataset_dir, image_id, 'image.tif')
          self.add_image("sketch", image_id=image_id, path=path)

    def load_mask(self, image_id):
        # Get mask directory from image path
        dataset_dir = '/tmp/traindata'
        
        info = self.image_info[image_id]
        
        # Get mask directory from image path
        mask_dirs = ['measurement_masks', 'parcel_number_masks']

        # Read mask files from .png image
        masks, labels = [], []
        mask_shape = (config.IMAGE_MAX_DIM, config.IMAGE_MIN_DIM)
        for i, name in enumerate(mask_dirs):
            path_dir = os.path.dirname(info['path'])
            mask_dir = os.path.join(dataset_dir, path_dir, name)
            if os.path.exists(mask_dir) and len(os.listdir(mask_dir)) != 0:
                for f in next(os.walk(mask_dir))[2]:
                    mask_img=np.zeros(mask_shape, dtype=np.bool)
                    m = np.load(os.path.join(mask_dir, f))
                    mask_img[m[0], m[1]] = 1
                    masks.append(mask_img)
                    labels.append(i + 1)
        
        if len(masks):
            masks = np.stack(masks, axis=-1)
            labels = np.asarray(labels, dtype=np.int32)
        else:
            masks = np.zeros((config.IMAGE_MAX_DIM, config.IMAGE_MIN_DIM, 0),
                             dtype=np.bool)
            labels = np.array([])

        return masks, labels

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "sketch":
            return info["id"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
image_ids = next(os.walk(dataset_dir))[1]

# Add images
random.shuffle(image_ids)

split = .8
image_ids_length = len(image_ids)
split_idx = int(image_ids_length * split)
train_ids, val_ids = image_ids[:split_idx], image_ids[split_idx:]

train_size, test_size = len(train_ids), len(val_ids)
print('Train set size: {train_size}.'.format(train_size=train_size))
print('Validation set size: {test_size}.'.format(test_size=test_size))

# Training dataset
dataset_train = SketchDataset()
dataset_train.load_sketch(train_ids)
dataset_train.prepare()

# Validation dataset
dataset_val = SketchDataset()
dataset_val.load_sketch(val_ids)
dataset_val.prepare()

In [ ]:
class SketchConfig(Config):

    # Give the configuration a recognizable name
    NAME = "sketch"

    BACKBONE='resnet50'

    USE_MINI_MASK = True

    # Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # background + measurement + parcel number

    # Use small images for faster training. Set the limits of the small side
    # the large side, which determines the image shape.
    IMAGE_MIN_DIM = 1216
    IMAGE_MAX_DIM = 1856

    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.7

    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.3
    
    # Percent of positive ROIs used to train classifier/mask heads
    ROI_POSITIVE_RATIO = 0.33

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 24, 32, 40)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 300

    # Epoch size should approximate train set
    STEPS_PER_EPOCH = train_size

    # Epoch size should approximate validation set
    VALIDATION_STEPS = test_size

config = SketchConfig()
config.display()

In [ ]:
# Load and display random samples
for image_id in np.random.choice(dataset_train.image_ids, 2):
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids,
                                dataset_train.class_names,
                                limit=config.NUM_CLASSES - 1)

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode='training', config=config, model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Local path to trained weights file
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
    # Download COCO trained weights from Releases if needed
    if not os.path.exists(COCO_MODEL_PATH):
        utils.download_trained_weights(COCO_MODEL_PATH)
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model_path = '/content/gdrive/My Drive/rcnn_snapshot/weights.h5'
    print("Loading weights from ", model_path)
    model.load_weights(model_path)

In [ ]:
# Train the head branches
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    '/content/gdrive/My Drive/rcnn_snapshot/weights.h5',
    monitor='val_loss', save_best_only=True, save_weights_only=True)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE,
            epochs=300, layers='all',
            custom_callbacks=[early_stopping, model_checkpoint])

In [ ]:
# Finetuning step
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    '/content/gdrive/My Drive/rcnn_snapshot/weights_fine.h5',
    monitor='val_loss', save_best_only=True, save_weights_only=True)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=300, layers='all',
            custom_callbacks=[early_stopping, model_checkpoint])

In [ ]:
class InferenceConfig(SketchConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Load trained weights
model_path = '/content/gdrive/My Drive/rcnn_snapshot/weights_fine.h5'
print("Loading weights from ", model_path)

model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask = \
    modellib.load_image_gt(dataset_val, inference_config, image_id,
                           use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

result = model.detect([original_image], verbose=1)[0]

masked_image = visualize.display_instances(
    original_image, result['rois'], result['masks'], result['class_ids'],
    dataset_val.class_names, result['scores'], ax=get_ax())